In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cp /content/drive/My\ Drive/fake_news_data.zip /content/
!unzip fake_news_data.zip
!rm fake_news_data.zip
!mv fake_news_data.csv fnc.csv
# %cp /content/drive/My\ Drive/best_stance_detector.h5 ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  fake_news_data.zip
  inflating: fake_news_data.csv      


In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
# import tensorflow_hub as hub
import tensorflow as tf
# import bert_tokenization as tokenization
import tensorflow.keras.backend as K
import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)

2.2.0


In [ ]:
df = pd.read_csv('fnc.csv')
df.dropna(subset=['labels', 'articleBody', 'Headline'], inplace=True)
print(len(df))
def sanitize(instance):
    if len(instance.split()) < 15:
        return ""
    return instance

df['Headline'] = df['Headline'].apply(sanitize)
df['articleBody'] = df['articleBody'].apply(sanitize)

df = df[df["Headline"] != ""]
df = df[df["articleBody"] != ""]

print(len(df))

206187
108179


In [ ]:
df['labels'].value_counts()

agree        71997
disagree     23013
unrelated    10742
discuss       2427
Name: labels, dtype: int64

In [ ]:
df_temp = df[df['labels'] != 'agree']
# df_temp = df_temp[df_temp['labels'] != "unrelated"]
# df_temp = df_temp[df_temp['labels'] != "discuss"]
df_temp2 = df[df['labels'] == 'agree']

df_temp2 = df_temp2.sample(25000)

df = pd.concat([df_temp, df_temp2])

df = df.sample(len(df))
# df = df.sample(10000)

In [ ]:
len(df)

61182

In [ ]:
HAS_ANS = False
training_sample_count = 12000 # 4000
training_epochs = 6 # 3
test_count = 20000
running_folds = 1 # 2
MAX_SEQUENCE_LENGTH = 256

from transformers import BertTokenizer, BertModel


MODEL_TYPE = 'bert-base-uncased'
MAX_SIZE = 200
BATCH_SIZE = 64

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)

In [ ]:
df_train = df[:training_sample_count*4]
print(df_train['labels'].value_counts())
one_hot = pd.get_dummies(df_train['labels'])
df_train = df_train.join(one_hot)

df_train.drop(['labels'],axis=1,inplace=True)

agree        19645
disagree     18042
unrelated     8398
discuss       1915
Name: labels, dtype: int64


In [ ]:
df_test = df[training_sample_count*4:]
print(df_test['labels'].value_counts())
one_hot = pd.get_dummies(df_test['labels'])
df_test = df_test.join(one_hot)

df_test.drop(['labels'],axis=1,inplace=True)

agree        5355
disagree     4971
unrelated    2344
discuss       512
Name: labels, dtype: int64


In [ ]:
test_df_y = df_test.copy()
del df_test['agree']
del df_test['disagree']
# del df_test['unrelated']
# del df_test['discuss']

df_sub = test_df_y.copy()

print(len(df),len(df_train),len(df_test))
display(df_train.head())
display(df_test.head())

61182 48000 13182


,Headline,articleBody,agree,disagree,discuss,unrelated
183009,anderson cooper just smacked kellyanne conway ...,"wednesday evening , trump surrogate kellyanne ...",0,1,0,0
199221,breaking : michael flynn agrees to testify on ...,you know how people have been wondering if mic...,0,1,0,0
20986,obama stonewalling on why he sent <number> pay...,leave it to the one and only andrew mccarthy t...,0,1,0,0
183880,abolish the exam in <number> ? listen to the l...,abolition of the middle test in <number> ? wha...,0,1,0,0
183865,henan : shooting incident in east station of s...,shooting incident at the south sanmen gorges h...,1,0,0,0


,Headline,articleBody,discuss,unrelated
73678,megyn kelly tears into ted cruz over ridiculou...,megyn kelly has been none too impressed with r...,0,0
133462,"gingers with it , a day to lose weight <number...","ginger slicing with it , taking two in the mor...",0,0
180236,xiong chenjin exposed esthetic wedding photos ...,xiong nai - jin is getting a wedding photo aga...,0,0
70550,conservative firebrand ann coulter destroys de...,there are some people you probably shouldn t c...,0,0
24536,obama administration denies congressman ' s cl...,a report from judicial watch stated that four ...,1,0


In [ ]:
output_categories = list(df_train.columns[[2, 3, 4, 5]])
input_categories = list(df_train.columns[[0, 1]])

if HAS_ANS:
    output_categories = list(df_train.columns[11:])
    input_categories = list(df_train.columns[[1,2,5]])
    

TARGET_COUNT = len(output_categories)

print('\ninput categories:\n\t', input_categories)
print('\noutput TARGET_COUNT:\n\t', TARGET_COUNT)
print('\noutput categories:\n\t', output_categories)


input categories:
	 ['Headline', 'articleBody']

output TARGET_COUNT:
	 4

output categories:
	 ['agree', 'disagree', 'discuss', 'unrelated']


In [ ]:
def _convert_to_transformer_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, str2, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1, str2,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy,
            truncation=True)
        
        input_ids =  inputs["input_ids"]
        input_masks = inputs['attention_mask']
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids_q, input_masks_q, input_segments_q = return_id(
        title, None, 'longest_first', max_sequence_length)
    
    input_ids_a, input_masks_a, input_segments_a = return_id(
        question, None, 'longest_first', max_sequence_length)
        
    return [input_ids_q, input_masks_q, input_segments_q,
            input_ids_a, input_masks_a, input_segments_a]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids_q, input_masks_q, input_segments_q = [], [], []
    input_ids_a, input_masks_a, input_segments_a = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q = instance.Headline, instance.articleBody
        try:
            ids_q, masks_q, segments_q, ids_a, masks_a, segments_a = \
            _convert_to_transformer_inputs(t, q, "", tokenizer, max_sequence_length)
        except:
            print(instance)
            var = input()
        input_ids_q.append(ids_q)
        input_masks_q.append(masks_q)
        input_segments_q.append(segments_q)
        input_ids_a.append(ids_a)
        input_masks_a.append(masks_a)
        input_segments_a.append(segments_a)
        
    return [np.asarray(input_ids_q, dtype=np.int32), 
            np.asarray(input_masks_q, dtype=np.int32), 
            np.asarray(input_segments_q, dtype=np.int32),
            np.asarray(input_ids_a, dtype=np.int32), 
            np.asarray(input_masks_a, dtype=np.int32), 
            np.asarray(input_segments_a, dtype=np.int32)]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

In [ ]:
outputs = compute_output_arrays(df_train, output_categories)
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

In [ ]:
def compute_spearmanr_ignore_nan(trues, preds):
    rhos = []
    for tcol, pcol in zip(np.transpose(trues), np.transpose(preds)):
        rhos.append(spearmanr(tcol, pcol).correlation)
    return np.nanmean(rhos)

def create_model():
    q_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_id = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_mask = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    q_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    a_atn = tf.keras.layers.Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32)
    
    config = BertConfig() # print(config) to see settings
    config.output_hidden_states = False # Set to True to obtain hidden states
    # caution: when using e.g. XLNet, XLNetConfig() will automatically use xlnet-large config
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased', config=config)
    
    # if config.output_hidden_states = True, obtain hidden states via bert_model(...)[-1]
    q_embedding = bert_model(q_id, attention_mask=q_mask, token_type_ids=q_atn)[0]
    a_embedding = bert_model(a_id, attention_mask=a_mask, token_type_ids=a_atn)[0]
    
    q = tf.keras.layers.GlobalAveragePooling1D()(q_embedding)
    a = tf.keras.layers.GlobalAveragePooling1D()(a_embedding)
    
    x = tf.keras.layers.Concatenate()([q, q])
    x = tf.keras.layers.Reshape((1, x.shape[-1]))(x)

    cnn = tf.keras.layers.Conv1D(64, 3, padding='same', activation='relu')(x)
    cnn = tf.keras.layers.MaxPooling1D(pool_size=1, strides=2)(cnn)
    cnn = tf.keras.layers.BatchNormalization()(cnn)

    lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64))(cnn)
    lstm = tf.keras.layers.Dropout(0.2)(lstm)
    
#     lstm = tf.keras.layers.Reshape((1, lstm.shape[-1]))(lstm)
    
    dense = tf.keras.layers.Dense(64,activation='relu')(lstm)
    x = tf.keras.layers.BatchNormalization()(x)
    
    x = tf.keras.layers.Dense(TARGET_COUNT, activation='softmax')(dense)
    x = tf.keras.layers.BatchNormalization()(x)

    model = tf.keras.models.Model(inputs=[q_id, q_mask, q_atn, a_id, a_mask, a_atn], outputs=x)
    
    return model

In [ ]:
model = create_model()
model.summary()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_25 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_27 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_29 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_bert_model_4 (TFBertModel)   ((None, 256, 768), ( 109482240   input_25[0][0]                   
____________________________________________________________________________________________

In [ ]:
# Evaluation Metrics
import sklearn
def print_evaluation_metrics(y_true, y_pred, label='', is_regression=True, label2=''):
    print('==================', label2)
    ### For regression
    if is_regression:
        print('mean_absolute_error',label,':', sklearn.metrics.mean_absolute_error(y_true, y_pred))
        print('mean_squared_error',label,':', sklearn.metrics.mean_squared_error(y_true, y_pred))
        print('r2 score',label,':', sklearn.metrics.r2_score(y_true, y_pred))
        #     print('max_error',label,':', sklearn.metrics.max_error(y_true, y_pred))
        return sklearn.metrics.mean_squared_error(y_true, y_pred)
    else:
        ### FOR Classification
        print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        # print('average_precision_score',label,':', sklearn.metrics.average_precision_score(y_true, y_pred))
        # print('balanced_accuracy_score',label,':', sklearn.metrics.balanced_accuracy_score(y_true, y_pred))
        print('accuracy_score',label,':', sklearn.metrics.accuracy_score(y_true, y_pred))
        print('f1_score',label,':', sklearn.metrics.f1_score(y_true, y_pred, average='weighted'))
        
        matrix = sklearn.metrics.confusion_matrix(y_true, y_pred)
        print(matrix)
        TP,TN,FP,FN = matrix[1][1],matrix[0][0],matrix[0][1],matrix[1][0]
        Accuracy = (TP+TN)/(TP+FP+FN+TN)
        Precision = TP/(TP+FP)
        Recall = TP/(TP+FN)
        F1 = 2*(Recall * Precision) / (Recall + Precision)
        print('Acc', Accuracy, 'Prec', Precision, 'Rec', Recall, 'F1',F1)
        return sklearn.metrics.accuracy_score(y_true, y_pred)

print_evaluation_metrics([1,0], [0.9,0.1], '', True)
print_evaluation_metrics([1,0], [1,1], '', False)

mean_absolute_error  : 0.09999999999999999
mean_squared_error  : 0.009999999999999998
r2 score  : 0.96
balanced_accuracy_score  : 0.5
accuracy_score  : 0.5
f1_score  : 0.3333333333333333
[[0 1]
 [0 1]]
Acc 0.5 Prec 0.5 Rec 1.0 F1 0.6666666666666666


0.5

In [ ]:
min_acc = 0
min_test = []
valid_preds = []
test_preds = []
best_model = False
best_lr = 1e-5
best_threshold = 0
for LR in [1e-5, 2e-5, 3e-5]:
    print('>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>')
    print('LR=', LR)
    gkf = GroupKFold(n_splits=5).split(X=df_train.Headline, groups=df_train.Headline)

    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold not in range(running_folds):
            continue
        train_inputs = [(inputs[i][train_idx])[:training_sample_count] for i in range(len(inputs))]
        train_outputs = (outputs[train_idx])[:training_sample_count]

        valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
        valid_outputs = outputs[valid_idx]

        print(np.array(train_inputs).shape, np.array(train_outputs).shape)
#         print(train_idx[:10], valid_idx[:10])

#         K.clear_session()
        model = create_model()
        optimizer = tf.keras.optimizers.Adam(learning_rate=LR)
        model.compile(loss='categorical_crossentropy', optimizer=optimizer)
        model.fit(train_inputs, train_outputs, epochs=3, batch_size=24)
        preds = model.predict(valid_inputs)
        valid_preds.append(np.argmax(preds, axis=1))
        valid_outputs = np.argmax(valid_outputs, axis=1)
        for xx in range(1):
            print(valid_outputs[:5])
            print(valid_preds[-1][:5])
            acc = print_evaluation_metrics(np.array(valid_outputs), np.array(valid_preds[-1]), 'on #'+str(xx+1), is_regression=False)
            if acc > min_acc:
                print('new acc >> ', acc)
                print('LR >> ', LR)
                min_acc = acc
                best_model = model
    #         min_test = model.predict(test_inputs)
    #         test_preds.append(min_test)
            print(' ')

best_model.save_weights('best_stance_detector_5.h5')

In [ ]:
# !rm /content/drive/My\ Drive/best_stance_detector.h5
# !rm best_stance_detector.h5
best_model.save_weights('best_stance_detector_5.h5')

In [ ]:
print("best acc >> ", min_acc)

In [ ]:
%cp best_stance_detector_5.h5 /content/drive/My\ Drive/